In [1]:
from transformers import CLIPModel, CLIPProcessor, CLIPConfig, CLIPTextConfig, CLIPVisionConfig
from transformers import pipeline
from PIL import Image
from typing import Type
import transformers
import requests
import torch
import copy
import sys
import os 

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import CLIPModel, CLIPSegModel, CLIPTextModel, CLIPTextModelWithProjection, CLIPVisionModelWithProjection, CLIPVisionModel
from transformers import CLIPSegTextModel, CLIPSegVisionModel

In [3]:
from transformers import AutoImageProcessor, AutoTokenizer, AutoBackbone, AutoProcessor, AutoModel, AutoFeatureExtractor, CLIPVisionConfig, CLIPTokenizer, CLIPTokenizerFast
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer, CLIPImageProcessor


In [ ]:
c = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
c2 = CLIPSegModel.from_pretrained("openai/clip-vit-base-patch32")
c3 = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
c4 = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
c5 = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
c6 = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
c7 = CLIPSegTextModel.from_pretrained("openai/clip-vit-base-patch32")
c8 = CLIPSegVisionModel.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
c

In [ ]:
equals = []
for x, y in zip(c.vision_model.parameters(), c6.parameters()):
    equals.append(x.equal(y))
all(equals)

In [ ]:
c6

In [4]:
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tok:CLIPTokenizerFast = CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
vis = CLIPImageProcessor()

In [5]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
img = Image.open("./save.png")
text = ["a photo of a cat", "a photo of a dog"]


In [6]:
pad_inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
outputs = clip(**pad_inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

tex_tok = tok(text=text, return_tensors="pt")
vis_img = vis(img, return_tensors="pt")
inputs = processor(text=text, images=img, return_tensors="pt")

outputs2 = clip(**tex_tok, **vis_img)

AttributeError: 'CLIPProcessor' object has no attribute 'eos_token'

In [9]:
inputs['input_ids']

tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]])

In [11]:
clip.

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [7]:
print(pad_inputs)
pad_inputs.keys()

{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]]), 'pixel_values': tensor([[[[ 0.5873,  0.5873,  0.6165,  ...,  0.0617,  0.0471, -0.0259],
          [ 0.5727,  0.5727,  0.6603,  ...,  0.1201,  0.0763,  0.0909],
          [ 0.5873,  0.5435,  0.6165,  ...,  0.0325,  0.1201,  0.0617],
          ...,
          [ 1.8719,  1.8573,  1.8719,  ...,  1.3902,  1.4340,  1.4194],
          [ 1.8281,  1.8719,  1.8427,  ...,  1.4486,  1.4340,  1.5070],
          [ 1.8573,  1.9011,  1.8281,  ...,  1.3756,  1.3610,  1.4486]],

         [[-1.3169, -1.3019, -1.3169,  ..., -1.4970, -1.4369, -1.4820],
          [-1.2418, -1.2718, -1.2268,  ..., -1.4369, -1.4669, -1.4519],
          [-1.2568, -1.3169, -1.2268,  ..., -1.4669, -1.4069, -1.4519],
          ...,
          [ 0.1239,  0.1089,  0.1239,  ..., -0.7016, -0.6865, -0.6865],
          [ 0.07

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [19]:
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)
inputs['pixel_values'].shape

In [20]:
vis_img['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [21]:
print(tex_tok['input_ids'].shape)
tex_tok['attention_mask'].shape

torch.Size([2, 7])


torch.Size([2, 7])

CLIPOutput(loss=None, logits_per_image=tensor([[18.8830, 20.3125]], grad_fn=<TBackward0>), logits_per_text=tensor([[18.8830],
        [20.3125]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 0.0148,  0.0070, -0.0234,  ..., -0.0508, -0.0438,  0.0033],
        [ 0.0087,  0.0258, -0.0387,  ..., -0.0547, -0.0242,  0.0112]],
       grad_fn=<DivBackward0>), image_embeds=tensor([[-4.2393e-02,  2.0662e-03, -4.9599e-04,  1.3198e-02, -9.5169e-03,
         -7.6227e-03,  6.7158e-03,  3.4872e-02,  1.5002e-02, -1.5889e-02,
          2.9736e-02,  3.9923e-03,  4.1415e-02,  1.3504e-02, -7.1151e-03,
         -2.6903e-03,  1.2073e-01, -1.4583e-02, -1.7149e-03,  2.4115e-02,
          1.0852e-01,  1.7230e-02,  1.7736e-03, -6.0261e-02,  6.5560e-03,
          3.9052e-02,  8.8093e-03, -1.2709e-02,  1.9773e-02, -2.2983e-02,
         -2.0093e-02,  2.9158e-02, -5.6362e-03,  9.1024e-03,  8.7693e-02,
          3.3201e-03,  1.3148e-02,  1.6944e-02, -1.5307e-02, -8.3911e-02,
          9.2842e-03, -7.8838e-03,  3.2

In [23]:
outputs.keys()

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [24]:
outputs['text_model_output']

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 1.0580, -0.9600,  1.0018,  ..., -0.5155, -0.1437, -1.9444],
         ...,
         [ 0.3059, -1.5037, -0.4022,  ..., -0.0224,  0.9105, -0.3916],
         [ 1.0118, -0.6701,  1.7742,  ..., -0.1556, -0.0250, -1.5062],
         [-0.5152,  0.1658,  0.8876,  ..., -0.0675, -0.4551, -1.7960]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 1.0580, -0.9600,  1.0018,  ..., -0.5155, -0.1437, -1.9444],
         ...,
         [ 0.3059, -1.5037, -0.4022,  ..., -0.0224,  0.9105, -0.3916],
         [-0.1433, -0.5163,  1.7099,  ..., -0.0795,  0.3609, -1.2437],
         [ 0.0426,  0.0189,  1.2740,  ..., -0.4217, -0.4393, -1.3016]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.5